# 생성한 요약문의 문법을 교정 (후처리)

- [나라 인포테크 맞춤법 검사기](http://164.125.7.61/speller/)를 이용하여 최종 제출 전 문법을 교정

In [ ]:
# 코랩과 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 라이브러리 불러오기
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
# 데이터 루트 설정
ROOT = '/content/drive/MyDrive/my_data/220320_kogpt/answer/'

In [ ]:
# 필요 데이터 불러오기 (맞춤법 검사를 할 문장 데이터)
data = pd.read_csv(ROOT+'answer_best.csv')
data.head(2)

,id,summary
0,xAqIHdvEZG,서울시여성가족재단은 세계 어린이날을 맞아 보육·교육기관에서 어린이가 겪는 성차별적 ...
1,zWSR9EQhrG,정부가 코로나19 여파로 직격탄을 맞은 사회적 취약계층과 소상공인 등에 대해 한시적...


In [ ]:
# 쉬운 처리를 위해 딕셔너리로 변환
for id, summary in data.values:
    print(id, summary)
    break
print(data.values[0])

xAqIHdvEZG 서울시여성가족재단은 세계 어린이날을 맞아 보육·교육기관에서 어린이가 겪는 성차별적 말과 행동을 양성평등 관점에서 바꾼 서울시 성평등 어린이사전을 발표했다.
['xAqIHdvEZG'
 '서울시여성가족재단은 세계 어린이날을 맞아 보육·교육기관에서 어린이가 겪는 성차별적 말과 행동을 양성평등 관점에서 바꾼 서울시 성평등 어린이사전을 발표했다.']


In [ ]:
## 테스트
# 맞춤법 검사 요청 (requests)
id, input_text = data.values[5]
response = requests.post('http://164.125.7.61/speller/results', data={'text1': input_text})
# 응답에서 필요한 내용 추출 (html 파싱)
html = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0]
# json_text를 dictionary로 변환
json_dict = json.loads(html)
# 교정된 문장 추출
output_text = input_text
for info in json_dict['errInfo']:
    output_text = output_text.replace(info['orgStr'], info['candWord'])
    
# 결과 비교
print("기존 문장:", input_text)
print("교정 문장:", output_text)

돌파감염은 돌파 감염은
접종일정을 접종 일정을
사람들 중에서 사람 중에서
경과된 지난
기존 문장: 돌파감염은 모든 접종일정을 완료한 사람들 중에서 14일 정도 경과된 다음에 발생하는 상황이며 백신 간의 비교는 바람직하지 않다.
교정 문장: 돌파 감염은 모든 접종 일정을 완료한 사람 중에서 14일 정도 지난 다음에 발생하는 상황이며 백신 간의 비교는 바람직하지 않다.


In [ ]:
## 실전 적용
for id, input_text in data.values:
    response = requests.post('http://164.125.7.61/speller/results', data={'text1': input_text})
    # 응답에서 필요한 내용 추출 (html 파싱)
    html = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0]
    # json_text를 dictionary로 변환
    try:
        json_dict = json.loads(html)
        # 교정된 문장 추출
        output_text = input_text
        for info in json_dict['errInfo']:
            output_text = output_text.replace(info['orgStr'], info['candWord'])
    except:
        output_text = input_text
    
    data.loc[data['id'] == id, 'summary'] = output_text

In [ ]:
# 파싱내용 확인
data.head(2)

,id,summary
0,xAqIHdvEZG,서울시여성가족재단은 세계 어린이날을 맞아 보육·교육기관에서 어린이가 겪는 성차별적 ...
1,zWSR9EQhrG,정부가 코로나19 여파로 직격탄을 맞은 사회적 취약계층과 소상공인 등에 대해 한시적...


In [ ]:
# 결과 저장
data.to_csv(ROOT+'answer_correction.csv', encoding='utf-8', index=False)